In [1]:
import os
from pathlib import Path
import shutil
import geopandas as gp
from shapely.geometry import mapping
import rioxarray as rxr
import xarray as xr
#import sys

In [3]:
IMG_DIR_IN = "/home/sandbox-cel/paraguay_lc/SegmentationData/time_series_vars"
POLY_DIR = "/home/downspout-cel/paraguay_lc/SegmentationData/user_train"
IMG_DIR_OUT = Path("/home/downspout-cel/paraguay_lc/SegmentationData/time_series_vars_crop")

In [84]:
##If passing from BASH:
#IMG_DIR_IN = sys.argv[1]
#POLY_DIR = sys.argv[2]
#IMG_DIR_OUT = sys.argv[3]

In [87]:
#IMG_DIR_OUT.mkdir(parents=True,exist_ok=True)

for gpkg in os.listdir(POLY_DIR):
    if 'grid' in gpkg:
        print('working on {}...'.format(gpkg))
        name = gpkg.split('_')
        yr = name[2][:4]
        cell = gpkg[:4]
        subdir_out = Path(os.path.join(IMG_DIR_OUT,name[0]))
        subdir_out.mkdir(parents=True,exist_ok=True)
        matchCellDir = os.path.join(IMG_DIR_IN,str(cell))
        subDirs = [f.name for f in os.scandir(matchCellDir) if f.is_dir()]
        for d in subDirs:
            subsubdir_out = Path(os.path.join(subdir_out,d))
            subsubdir_out.mkdir(parents=True,exist_ok=True)
            subsubdir_in = os.path.join(matchCellDir,d)
            for img in os.listdir(subsubdir_in):
                image = rxr.open_rasterio(os.path.join(subsubdir_in,img),
                                 masked=True).squeeze()
                crop_extent = gpd.read_file(os.path.join(POLY_DIR,gpkg))
                img_clipped = image.rio.clip(crop_extent.geometry.apply(mapping))
                img_clipped.rio.to_raster(os.path.join(subsubdir_out,img))
                ##To copy file without cropping, coment-out two lines above and use:
                #shutil.copyfile(os.path.join(subsubdir_in,img), os.path.join(subsubdir_out,img))

In [ ]:
###Add missing grid field to gpkg:

In [5]:
POLY_DIR = "/home/downspout-cel/paraguay_lc/SegmentationData/user_train"

import geopandas as gpd
import fiona

for item in os.listdir(POLY_DIR):
    if 'grid' in item: 
        gpkg = os.path.join(POLY_DIR,item)
        print(gpkg)
        src = gpd.read_file(gpkg)
        src['grid']=0
        print(src)
        src.to_file(os.path.join(POLY_DIR,'out',item), driver="GPKG")

/home/downspout-cel/paraguay_lc/SegmentationData/user_train/096002_grid_2018.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        32  708-870         1  960.0  2022-07-27T17:36:45  cell_1668  960002   

               Name  grid                                           geometry  
0  960002_grid_2018     0  MULTIPOLYGON (((435550.000 695270.000, 435550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/097803_grid_2018.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        51  708-869         1  978.0  2022-07-27T15:04:54  cell_1723  978003   

               Name  grid                                           geometry  
0  978003_grid_2018     0  MULTIPOLYGON (((429550.000 673270.000, 429550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/094201_grid_2018.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        23  708-871         1 

/home/downspout-cel/paraguay_lc/SegmentationData/user_train/097402_grid_2018.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        59  704-868         1  974.0  2022-08-05T15:42:42  cell_1750  974002   

               Name  grid                                           geometry  
0  974002_grid_2018     0  MULTIPOLYGON (((357550.000 660270.000, 357550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/102903_grid_2021.gpkg
   OBJECTID PlanetID  Priority     UNQ                Start     CellID  \
0        90  704-866         1  1029.0  2022-08-08T17:02:08  cell_1853   

   region              Name  grid  \
0  102903  102903_grid_2018     0   

                                            geometry  
0  MULTIPOLYGON (((357550.000 607270.000, 357550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/102804_grid_2021.gpkg
   OBJECTID PlanetID  Priority     UNQ                Start     CellID  \
0        91  702-865     

/home/downspout-cel/paraguay_lc/SegmentationData/user_train/099403_grid_2021.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        68  706-867         1  994.0  2022-07-25T16:07:36  cell_1779  994003   

               Name  grid                                           geometry  
0  994003_grid_2018     0  MULTIPOLYGON (((399550.000 642270.000, 399550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/090901_grid_2018.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0         2  707-874         1  909.0  2022-07-29T16:35:13  cell_1542  909001   

               Name  grid                                           geometry  
0  909001_grid_2018     0  MULTIPOLYGON (((424550.000 756270.000, 424550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/099504_grid_2021.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        66  707-868         1 

/home/downspout-cel/paraguay_lc/SegmentationData/user_train/092501_grid_2018.gpkg
   OBJECTID PlanetID  Priority    UNQ Start     CellID  region  \
0         8  709-873         1  925.0  None  cell_1587  925001   

               Name  grid                                           geometry  
0  925001_grid_2018     0  MULTIPOLYGON (((441550.000 735270.000, 441550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/102801_grid_2021.gpkg
   OBJECTID PlanetID  Priority     UNQ                Start     CellID  \
0        87  702-866         1  1028.0  2022-08-16T22:42:03  cell_1841   

   region             Name  grid  \
0   12801  12801_grid_2018     0   

                                            geometry  
0  MULTIPOLYGON (((335550.000 611270.000, 335550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/100802_grid_2018.gpkg
   OBJECTID PlanetID  Priority     UNQ                Start     CellID  \
0        80  701-867         1  1008.0  2022-08-08T14:51:

/home/downspout-cel/paraguay_lc/SegmentationData/user_train/092406_grid_2021.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        14  707-872         1  924.0  2022-07-06T14:15:30  cell_1605  924006   

               Name  grid                                           geometry  
0  924006_grid_2018     0  MULTIPOLYGON (((418550.000 726270.000, 418550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/095802_grid_2018.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        29  706-870         1  958.0  2022-07-11T14:00:59  cell_1662  958002   

               Name  grid                                           geometry  
0  958002_grid_2018     0  MULTIPOLYGON (((393550.000 697270.000, 393550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/097801_grid_2021.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        45  708-869         1 

/home/downspout-cel/paraguay_lc/SegmentationData/user_train/097701_grid_2018.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        55  707-869         1  977.0  2022-07-20T16:13:59  cell_1743  977001   

               Name  grid                                           geometry  
0  977001_grid_2018     0  MULTIPOLYGON (((415550.000 662270.000, 415550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/095904_grid_2018.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        40  707-870         1  959.0  2022-07-13T15:38:14  cell_1689  959004   

               Name  grid                                           geometry  
0  959004_grid_2018     0  MULTIPOLYGON (((418550.000 687270.000, 418550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/092407_grid_2021.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        15  707-872         1 

/home/downspout-cel/paraguay_lc/SegmentationData/user_train/092407_grid_2018.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        15  707-872         1  924.0  2022-07-06T15:55:22  cell_1610  924007   

               Name  grid                                           geometry  
0  924007_grid_2018     0  MULTIPOLYGON (((410550.000 723270.000, 410550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/094002_grid_2021.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        21  706-871         1  940.0  2022-06-27T14:11:04  cell_1632  940002   

               Name  grid                                           geometry  
0  940002_grid_2018     0  MULTIPOLYGON (((397550.000 712270.000, 397550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/095904_grid_2021.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        40  707-870         1 

/home/downspout-cel/paraguay_lc/SegmentationData/user_train/093901_grid_2021.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        20  705-871         1  939.0  2022-08-15T22:10:16  cell_1628  939001   

               Name  grid                                           geometry  
0  939001_grid_2018     0  MULTIPOLYGON (((381550.000 714270.000, 381550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/097802_grid_2021.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        46  708-869         1  978.0  2022-07-27T16:26:21  cell_1708  978002   

               Name  grid                                           geometry  
0  978002_grid_2018     0  MULTIPOLYGON (((431550.000 678270.000, 431550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/096003_grid_2021.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        34  708-870         1 

/home/downspout-cel/paraguay_lc/SegmentationData/user_train/097301_grid_2021.gpkg
   OBJECTID PlanetID  Priority    UNQ Start     CellID  region  \
0        54  702-869         1  973.0  None  cell_1735  973001   

               Name  grid                                           geometry  
0  973001_grid_2018     0  MULTIPOLYGON (((338550.000 666270.000, 338550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/099502_grid_2018.gpkg
   OBJECTID PlanetID  Priority    UNQ                Start     CellID  region  \
0        62  706-868         1  995.0  2022-07-25T14:41:17  cell_1758  995002   

               Name  grid                                           geometry  
0  995002_grid_2018     0  MULTIPOLYGON (((405550.000 656270.000, 405550....  
/home/downspout-cel/paraguay_lc/SegmentationData/user_train/100801_grid_2018.gpkg
   OBJECTID PlanetID  Priority     UNQ                Start     CellID  \
0        74  701-867         1  1008.0  2022-08-08T14:49:30  cell_17

In [6]:
##Copy new images back into original file. NOte overwrite -- may want to copy folder first
for img in os.listdir(os.path.join(POLY_DIR,'out')):
    shutil.move(os.path.join(POLY_DIR,'out',img), os.path.join(POLY_DIR,img))